# Creating Dataset

In [ ]:
import numpy as np
import os
import cv2

### Setting Image size

In [ ]:
x_image = 50
y_image = 50

### Checking/Creating Folder Directory

In [ ]:
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

In [ ]:
def store_images(g_id):
    
    # Total images per class
    total_images = 1200
    
    # Initializing Camera
    cap = cv2.VideoCapture(0)
    
    # Creating Directory
    name = ("Gestures/" + str(g_id))
    create_folder(name)
    
    
    pic_no = 0
    flag_start_capturing = False
    frames = 0
    while True:
        # Read image
        ret, frame = cap.read()
        # Flip Image
        frame = cv2.flip(frame, 1)
        # BGR2HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # Extracting color from image
        mask = cv2.inRange(hsv, np.array([2, 25, 78]), np.array([25, 150, 255]))
        # Merging two images
        res = cv2.bitwise_and(frame, frame, mask=mask)
        # BGR2Gray
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        # Blurring image
        median = cv2.GaussianBlur(gray, (5,5), 0)
        
        # Dilation + Opening {Increasing area of object}
        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)
        
        # Threshold image
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)
        # ROI
        x, y, w, h = 300, 50, 350, 350
        thresh = thresh[y:y + h, x:x + w]
        # Finding Contours
        contours, hierarchy = cv2.findContours(thresh.copy()  ,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if flag_start_capturing == True:
                x1, y1, w1, h1 = cv2.boundingRect(contour)
                pic_no += 1
                image_1 = thresh[y1:y1 + h1, x1:x1 + w1]
                if w1 > h1:
                    image_1 = cv2.copyMakeBorder(image_1, int((w1 - h1) / 2), int((w1 - h1) / 2), 0, 0,
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                elif h1 > w1:
                    image_1 = cv2.copyMakeBorder(image_1, 0, 0, int((h1 - w1) / 2), int((h1 - w1) / 2),
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                final = cv2.resize(image_1, (x_image, y_image))
                cv2.putText(frame, "Capturing...", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
                # Saving Image
                cv2.imwrite(str("Gestures/" + g_id) +  "/" + str(pic_no) + ".jpg", final)
        
        # Drawing Rectangle around ROI
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Putting textof image number
        cv2.putText(frame, str(pic_no), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (127, 127, 255))
        
        cv2.imshow("Capturing gesture", frame)
        cv2.imshow("thresh", thresh)
        
        # Taking input key
        keypress = cv2.waitKey(1)
        
        # If input key is "c"
        # Start capturing
        if keypress == ord('c'):
            if flag_start_capturing == False:
                flag_start_capturing = True
            else:
                flag_start_capturing = False
                frames = 0
                
        # If input key is "q"
        if keypress == ord('q'):
            break
            
        if flag_start_capturing == True:
            frames += 1
        if pic_no == total_images:
            break
        
    cap.release
    cv2.destroyAllWindows()

In [ ]:
g_id = input("Enter the gesture no. : ")
store_images(g_id)